In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import ast

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [16]:
def calculate_com(landmarks):
    """
    Calcula el centro de masa (promedio de las posiciones de las articulaciones clave).
    Utiliza las caderas y hombros para el cálculo.
    """
    # Puntos clave: Caderas y Hombros
    left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_HIP].y,
                         landmarks[mp_pose.PoseLandmark.LEFT_HIP].z])

    right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y,
                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP].z])

    left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].z])

    right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].z])

    # Calcula el centro de masa como el promedio de los puntos clave
    com = (left_hip + right_hip + left_shoulder + right_shoulder) / 4
    return com


def calculate_torso_orientation(landmarks):
    """
    Calcula la orientación del torso usando los hombros y caderas.
    """
    # Puntos clave: Hombros y Caderas
    left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].z])

    right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].z])

    left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_HIP].y,
                         landmarks[mp_pose.PoseLandmark.LEFT_HIP].z])

    right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y,
                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP].z])

    # Vector hombros (dirección entre los hombros)
    shoulder_vector = right_shoulder - left_shoulder

    # Centro de las caderas
    hip_center = (left_hip + right_hip) / 2

    # Vector torso (dirección entre los hombros y el centro de las caderas)
    torso_vector = hip_center - (left_shoulder + right_shoulder) / 2

    # Producto cruzado para obtener la normal al plano del torso
    torso_normal = np.cross(shoulder_vector, torso_vector)

    # Normalización del vector normal
    torso_normal = torso_normal / np.linalg.norm(torso_normal)

    return torso_normal

def process_videos(video_directory: str, output_file: str, target: int, frames_per_second: int = 6):
    video_data = []  # Para almacenar los datos de todos los videos
    video_filenames = [f for f in os.listdir(video_directory) if f.endswith(".mp4") or f.endswith(".avi")]

    for video_filename in tqdm(video_filenames, desc="Procesando videos"):
        video_path = os.path.join(video_directory, video_filename)
        #print(f"Procesando video: {video_filename}")

        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        prev_landmarks = None
        prev_velocity = None
        video_features = []
        video_poses = []
        
        frame_skip = int(fps // frames_per_second)
        frame_count = 0
        
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break
        
            frame_count += 1
            if frame_count % frame_skip != 0:
                continue  # Saltar este frame
            frame_count = 0
            
            # Procesar la imagen con MediaPipe Pose
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
        
            if results.pose_world_landmarks:
                landmarks = results.pose_world_landmarks.landmark
                joint_index = 0  # Índice del punto a analizar (ej. 0 = Nariz, 23 = Cadera izquierda, etc.)
        
                if prev_landmarks is not None:
                    # Obtener coordenadas actuales y anteriores
                    x, y, z = landmarks[joint_index].x, landmarks[joint_index].y, landmarks[joint_index].z
                    x_prev, y_prev, z_prev = prev_landmarks[joint_index]
        
                    # Desplazamiento
                    dx, dy, dz = x - x_prev, y - y_prev, z - z_prev
                    displacement = np.sqrt(dx**2 + dy**2 + dz**2)
        
                    # Velocidad (por frame, no por tiempo)
                    velocity_x, velocity_y, velocity_z = dx, dy, dz
                    velocity = np.sqrt(velocity_x**2 + velocity_y**2 + velocity_z**2)
        
                    # Aceleración (cambio de velocidad por frame)
                    if prev_velocity is not None:
                        acceleration_x = velocity_x - prev_velocity[0]
                        acceleration_y = velocity_y - prev_velocity[1]
                        acceleration_z = velocity_z - prev_velocity[2]
                        acceleration = np.sqrt(acceleration_x**2 + acceleration_y**2 + acceleration_z**2)
                    else:
                        acceleration_x, acceleration_y, acceleration_z, acceleration = 0, 0, 0, 0
        
                    tangent_angle = np.arctan2(dy, dx) * (180 / np.pi)
                    com = calculate_com(landmarks)
                    com_x, com_y, com_z = com
                    torso_normal = calculate_torso_orientation(landmarks)
        
                    frame_features = {
                        "displacement": displacement,
                        "velocity": velocity,
                        "acceleration": acceleration,
                        "tangent_angle": tangent_angle,
                        "com": (com_x, com_y, com_z),
                        "torso_normal": torso_normal
                    }
                    video_features.append(frame_features)
                    prev_velocity = (velocity_x, velocity_y, velocity_z)
                
                else:
                    frame_features = {
                        "displacement": 0.0,
                        "velocity": 0.0,
                        "acceleration": 0.0,
                        "tangent_angle": 0.0,
                        "com": (0.0, 0.0, 0.0),
                        "torso_normal": [0.0, 0.0, 0.0]
                    }
                    video_features.append(frame_features)
                prev_landmarks = [(lm.x, lm.y, lm.z) for lm in landmarks]
                video_poses.append(prev_landmarks)
            
        cap.release()

        video_data.append({
            "video_filename": video_filename,
            "features": video_features,
            "poses": video_poses,
            "target": target
        })

    df_rows = []

    for video in video_data:
        filename = video["video_filename"]
        features = video["features"]
        poses = video["poses"]
        target = video["target"]
    
        # Convertir cada frame de poses a un solo vector [x1, y1, z1, ..., xN, yN, zN]
        poses_sec = []
        for pose_un in poses:
            flat_pose = []
            for x, y, z in pose_un:
                flat_pose.extend([x, y, z])
            poses_sec.append(flat_pose)
    
        # Opcional: convertir cada diccionario de features a un vector ordenado
        features_sec = []
        for feat in features:
            features_sec.append([
                feat["displacement"],
                feat["velocity"],
                feat["acceleration"],
                feat["tangent_angle"],
                feat["com"][0], feat["com"][1], feat["com"][2],
                feat["torso_normal"][0], feat["torso_normal"][1], feat["torso_normal"][2]
            ])
    
        df_rows.append({
            "video_filename": filename,
            "poses_sec": poses_sec,
            "features": features_sec,
            "target": target
        })
    
    # Crear el DataFrame
    df = pd.DataFrame(df_rows)
    
    # Guardar como CSV
    df.to_csv(f"{output_file}.csv", index=False)
    
    print(f"Datos guardados en {output_file}.csv")

In [17]:
process_videos(video_directory="C:/Users/bryan/OneDrive/Documentos/Tesis/fall_detection/data/GMDCSA24/Fall", output_file="fall", target=1)

process_videos(video_directory="C:/Users/bryan/OneDrive/Documentos/Tesis/fall_detection/data/GMDCSA24/ADL", output_file="adl", target=0)

Procesando videos: 100%|██████████| 67/67 [02:44<00:00,  2.45s/it]


Datos guardados en fall


Procesando videos: 100%|██████████| 81/81 [03:59<00:00,  2.95s/it]


Datos guardados en adl


In [12]:
def unir_csvs(ruta_csv1, ruta_csv2, ruta_salida):
    # Leer los CSV
    df1 = pd.read_csv(ruta_csv1)
    df2 = pd.read_csv(ruta_csv2)

    # Verificar que tienen las mismas columnas
    if list(df1.columns) != list(df2.columns):
        raise ValueError("Los archivos CSV no tienen las mismas columnas.")

    # Unir los DataFrames
    df_combinado = pd.concat([df1, df2], ignore_index=True)

    # Guardar el resultado
    df_combinado.to_csv(ruta_salida, index=False)
    print(f"Archivo combinado guardado en: {ruta_salida}")

In [13]:
def clean_csv(file:str, outfile:str):
    df = pd.read_csv(file)
    
    # Convertir las cadenas a listas reales usando ast.literal_eval
    df["poses_sec"] = df["poses_sec"].apply(ast.literal_eval)
    df["features"] = df["features"].apply(ast.literal_eval)
    
    # Obtener la longitud máxima de poses y features
    max_len_poses = max(len(seq) for seq in df["poses_sec"])
    max_len_features = max(len(seq) for seq in df["features"])
    
    # Aplicar padding a cada fila
    for idx, row in df.iterrows():
        #print(f"Video {idx}: poses_len={len(row['poses_sec'])}, features_len={len(row['features'])}")
        poses_seq = row["poses_sec"]
        features_seq = row["features"]
    
        # Padding de poses
        if len(poses_seq) < max_len_poses:
            num_pose_dims = len(poses_seq[0]) if poses_seq else 99  # 33 puntos x 3 coords
            padding_poses = [[0.0] * num_pose_dims] * (max_len_poses - len(poses_seq))
            poses_seq.extend(padding_poses)
    
        # Padding de features
        if len(features_seq) < max_len_features:
            num_feat_dims = len(features_seq[0]) if features_seq else 10  # 10 características
            padding_feats = [[0.0] * num_feat_dims] * (max_len_features - len(features_seq))
            features_seq.extend(padding_feats)
    
        # Guardar las secuencias de nuevo en el DataFrame
        df.at[idx, "poses_sec"] = poses_seq
        df.at[idx, "features"] = features_seq
    
    # Guardar el nuevo DataFrame
    df.to_csv(outfile, index=False)
    
    print("✅ Secuencias normalizadas y guardadas.")

In [14]:
def verify_csv(file:str):
    df = pd.read_csv(file)
    
    df["poses_sec"] = df["poses_sec"].apply(ast.literal_eval)
    df["features"] = df["features"].apply(ast.literal_eval)
    
    # Revisar la longitud de cada secuencia
    lengths_poses = df["poses_sec"].apply(len)
    lengths_features = df["features"].apply(len)
    
    # Verificar si todas tienen la misma longitud
    consistent_poses = lengths_poses.nunique() == 1
    consistent_features = lengths_features.nunique() == 1
    
    print("✅ Verificación de secuencias:")
    print(f"- Todas las secuencias de 'poses_sec' tienen la misma longitud: {consistent_poses}")
    print(f"- Todas las secuencias de 'features' tienen la misma longitud: {consistent_features}")
    print(f"  Longitud común de poses_sec: {lengths_poses.iloc[0]}")
    print(f"  Longitud común de features: {lengths_features.iloc[0]}")

In [18]:
unir_csvs("adl.csv", "fall.csv", "combined.csv")
clean_csv("combined.csv", "combined_padded.csv")
verify_csv("combined_padded.csv")

Archivo combinado guardado en: combined.csv
Video 0: poses_len=60, features_len=60
Video 1: poses_len=48, features_len=48
Video 2: poses_len=49, features_len=49
Video 3: poses_len=73, features_len=73
Video 4: poses_len=80, features_len=80
Video 5: poses_len=59, features_len=59
Video 6: poses_len=63, features_len=63
Video 7: poses_len=56, features_len=56
Video 8: poses_len=76, features_len=76
Video 9: poses_len=90, features_len=90
Video 10: poses_len=81, features_len=81
Video 11: poses_len=43, features_len=43
Video 12: poses_len=70, features_len=70
Video 13: poses_len=76, features_len=76
Video 14: poses_len=75, features_len=75
Video 15: poses_len=70, features_len=70
Video 16: poses_len=65, features_len=65
Video 17: poses_len=67, features_len=67
Video 18: poses_len=38, features_len=38
Video 19: poses_len=65, features_len=65
Video 20: poses_len=24, features_len=24
Video 21: poses_len=36, features_len=36
Video 22: poses_len=58, features_len=58
Video 23: poses_len=92, features_len=92
Video 